# SQL

## DB의 이해
- 데이터베이스 (Database)
    - 엑셀과 유사한 구조
    - 관계형 데이터베이스 (RDB, Relational Database) 
    - 예) Oracle, mysql, postgres
    - mysql database server의 계층 구조
        - mysql database server > 스키마 n개 > 테이블 n개
    - CRUD: 추가, 조회, 수정, 삭제를 의미하는 IT 전반의 용어 (create, read, update, delete)
- IP 프로토콜의 한계
    - 패킷을 받을 대상서버가 불능이거나, 중간에 패킷이 소실될 수가 있음 (비연결성)
    - 패킷의 순서가 보장되지 않음

## DB 사용하기 (MySQL 밒 WorkBench 설치)
- 터미널에서 다음 입력:
    - docker run --name board_mysql -e MYSQL_ROOT_PASSWORD=1234 -d -p 3406:3306 mysql
    - 원래는 "docker run --name board_mysql -e MYSQL_ROOT_PASSWORD=1234 -d -p 3306:3306 mysql"
    인데 학원에서는 공용컴퓨터이다 보니 다른 사람이 그 컴퓨터에 docker가 아닌 컴퓨터에 이미 깔아놓아서 같은 이름으로 설치하지 못하게 하는 것 --> 그래서 3306을 3406으로 변환해서 설치했음
    - 3406으로 했어도 껍데기만 3406으로 저장해놨을 뿐 안에서는 3306으로 인식 될 것임. 왜냐하면 mysql개발자가 최초에 만들 때 3306으로 만들었기 때문임
- docker 컨테이너 내부로 접속
    - docker ps -a
    - docker start 컨테이너ID
    - docker exec -it 컨테이너ID(또는 컨테이너명) mysql -u root -p 1234*****
        - 위 명령에서 `-it`는 대화형(interactive) 모드로 실행하고 터미널에 접속하도록 하는 옵션
        - 여기서 -p 다음을 입력하지 않고 enter 누르기 
        - 왜냐하면 history보면 다른 사람이 볼 수도 있기 때문
        - *****빼고 enter 누른 다음 enter password: 뜨면 1234 입력하고 enter
    - -u : user
    - -p : password
- docker exec -it board_mysql mysql -u root -p

## DB의 `index` 
- (면접 때 많이 물어봄 - index에 아는 것에 대하여 설명해보세요)
- 검색의 효율성, 색인(목차), B-Tree 자료구조 
- 명령어:
    - SHOW INDEX FROM 테이블명; -> 조회
    - CREATE INDEX 인덱스명 ON 테이블명(컬럼1, 컬럼2, ...); -> 생성
        - 예) CREATE INDEX my_index ON author(name); -> 생성
- Cardinality (차수): 컬럼 내의 데이터의 다양성/종류 개수
    - cardinality가 높은 컬럼에 index를 걸어야 효율적

- INDEX는 자주 사용되는 필드 값으로 만들어진 원본 테이블의 사본, 색인과 목차로 이해하면 됨
- MySQL은 데이터를 검색할 떄 첫 번째 필드부터 차례대로 테이블 전체를 검색
- 인덱스를 사용하면 테이블 전체를 읽지 않아도 된다
- 자주 사용하는 컬럼에 INDEX를 건다
    - 예) INDEX가 없이 테이블 전체를 읽어야 할 때
        - id = 897 번째의 데이터를 조회하기 위해서는 897번의 check가 필요
    - 예) id에 INDEX를 걸어 목차를 만들었을 때
        - id = 897 번째의 데이터를 조회하기 위해서는 8 + 9 + 7 = 24번이면 족하다
- Seq_in_index
    - INDEX는 조회 WHERE 조건에 INDEX 키를 조건으로 걸어줘야 INDEX 페이지를 검색
        - SELECT * FROM author WHERE id = 1;
        - SELECT * FROM author WHERE id = 1 AND name = 'hong';
            - name에 INDEX가 없다면 id INDEX 페이지르 참조
            - name에도 별도로 INDEX가 있다면, mysql엔진에서 최적의 알고리즘 실행
            - id, name에 동시에 index가 걸려있다면 사전에 2key를 통해 정렬된 페이지에서 참조
    - 예) 두 가지로 조회하는 경우가 많으면 두 가지를 한꺼번에 INDEX 를 거는 것이 효율적이다 - 두 가지를 조건으로 걸 때
        - 예) create index my_index2 on author (name, role);
            - Seq_in_index 에 name과 role이 각각 1, 2이다
            - 이 경우, SELECT * FROM author WHERE name = 'hong' AND role = 'user'; 하면 my_index2를 타서 조회하므로 시간적으로 효율적이다
- PK, UNIQUE, FK 컬럼은 자동으로 INDEX가 생성된다
- 실무에서 날짜에 INDEX를 걸면 좋을 것 - unique하기 때문에, 날짜를 조회하는 경우가 많기 때문에 - 성능이 빨라질 것임, 성능이 높아질 것임


## Transaction
- 트렌젝션은 데이터베이스에서 수행되는 하나 이상의 작업(쿼리)을 논리적으로 묶어서 실행하는 단위
- 모든 작업이 정상완료 되어야 작업의 결과를 확정, 하나라도 실패 시 모두 rollback되어 이전 상태로 복구
- mysql은 별도의 TRANSACTION 구문을 써주지 않으면 auto commit 또는 auto rollback 한다 - oracle은 auto commit아닌 것 같음
- 문법:
    ```mysql
    START TRANSACTION;

    INSERT INTO author ...
    CASE
    WHEN ROW_COUNT() = 0 THEN ROLLBACK
    END

    UPDATE post SET ...
    CASE
    WHEN ROW_COUNT() = 0 THEN ROLLBACK
    END

    COMMIT; [만약에 위 쿼리 중 하나에 문제가 있으면 rollback;]
    ```
- 실무에서는 sql문 쿼리로 transaction 짜는 경우는 많지 않고, 일반적으로 웹 프레임워크에서 transaction 잡고 쿼리를 실행한다 (django)
    - 예)
        def orders():
            insert into author ...
            update post ...
- 트랜잭션의 4가지 특성: ACID 
    - Atomicity (원자성) **중요**
        - TRANSACTION 실행 시 모두 실행되던지 모두 실행되지 않도록 하던지를 보장
        - 송금시 한쪽만 돈이 빠져나가서는 안 됨
    - Consistenty (일관성)
        - TRANSACTION 완료 시
        - 잔액의 합계가 일정
    - Isolation (고립성)
        - TRANSACTION 실행 중 다른 TRANSACTION이 끼어들 수 없음을 보장
    - Durability (영속성)
        - 물리적 데이터 저장 보장
- 프레임워크마다 다르기 때문에, 프로그램 내에서 별도로 TRANSACTION 처리를 해줘야 하는 경우가 많다
    - 예) 예외 발생 시 rollback 처리
        try xxxx:
            (쿼리 - orm기술)
            author.save();
            post.get();
            commit;
        except Exception e:
            rollback

## SQL 명령어
- 모든 SQL 반드시 ;으로 끝맺음
- 데이터베이스 == 스키마
- 목록조회
    - SHOW databases;
        - 데이터베이스(스키마)의 목록조회
    - SHOW tables;
        - 테이블의 목록조회
- 데이터베이스(스키마) 생성
    - CREATE DATABASE 스키마명
- 데이터베이스(스키마) 삭제
    - DROP DATABASE 스키마명
- 테이블 생성
    - USE 스키마명
    - CREATE TABLE 테이블명(id INT);
- 테이블의 spec(속성)/구조 조회
    - DESCRIBE 테이블명
    - 또는 DESC 테이블명
- 테이블 삭제
    - DROP TABLE 테이블명
    - if 문 사용 가능:
        - DROP TABLE IF EXISTS 테이블명;
- 테이블 값만 전부 삭제
    - TRUNCATE TABLE 테이블명
    - = DELETE FROM 테이블명 (전부 다 지울때)
    - TRUNCATE가 DELETE보다 속도 더 빠름
- 자동으로 숫자 올림순으로 매겨짐 
    - AUTO_INCREMENT
        - 테이블 생성/수정시 필드타입 뒤에 씀
        - 이것을 사용할 시 이 테이블의 PK는 무조건 이 필드만 할 수 있음.
        - 예) CREATE TABLE author(id INT auto_increment, ...
    - AUTO_INCREMENT 다시 세팅 시
        - 만약 테이블을 만들고 데이터 입력 후 삭제한 후 id value 숫자를 세팅하고 싶을 때:
        - DELETE FROM author WHERE id >= 10;
        - AUTO_INCREMENT 를 다시 세팅해주면 된다;
        - ALTER TABLE author AUTO_INCREMENT = 8;
- 날짜와 시간
    - DATETIME -> 자동생성 됨
        - 테이블 생성/수정시 필드타입에 씀
        - 예) CREATE TABLE author(created_at DATETIME);
        - 'YYYY-MM-DD HH:MM:SS'로 출력됨
            - 예) '2023-06-05 시간:분:초'
    - DATETIME INSERT 방법
        - 테이블 생성/수정시 필드타입에 씀
        - 덮어쓰기: '2022-06-05' 하면 시간은 자동으로 00:00:00으로 됨, '2022-06-05 05:00:00'
- min와 max
    - MIN(컬럼명) -> 컬럼 안의 가장 작은 값 출력
    - MAX(컬럼명) -> 컬럼 안의 가장 큰 값 출력
    - 예) SELECT MIN(created_at) FROM author; -> 가장 먼저 만들어진 날짜시간 출력
- 제약조건
    - 데이터를 입력받을 때 실행되는 검사 규칙
    - CREATE 문으로 테이블을 생성 또는 ALTER 문으로 필드를 추가할 때 설정
    - PK, FK, UNIQUE 제약조건을 걸게 되면 자동으로 index가 생성됨
    - 종류:
        - NOT NULL: 비면 안 되는 것
            - 테이블 생성/수정시 필드타입 뒤에 씀
            - 여러 필드에 설정 가능
            - 예) CREATE TABLE author(email VARCHAR(40) NOT NULL, ...);
        - UNIQUE: 유일무이해야하는 옵션
            - 테이블 생성/수정시 필드타입 뒤에 씀
            - 여러 필드에 설정 가능
        - PRIMARY KEY: 
            - 테이블 생성/수정시 맨 끝에 씀
            - 테이블당 오직 하나의 필드에만 설정 가능 (unlike NOT NULL와 UNIQUE)
            - 예)CREATE TABLE author(id BIGINT, name VARCHAR(20), ... , PRIMARY KEY(id));
        - FOREIGN KEY (외래키): 기준이 되는 다른 테이블의 내용을 참조해서 레코드가 입력, 하나의 테이블을 다른 테이블에 의존하게 만드는 것
            - 테이블 생성/수정시 맨 끝에 씀
            - 다른 테이블의 필드는 반드시 UNIQUE나 PK 제약 조건이어야 함
            - ... , FOREIGN KEY (필드명) REFERENCES 다른테이블명(다른테이블의필드명)
            - ON DELETE 옵션이 별도로 없으면 RESTRICT(삭제안됨)가 DEFAULT
            - 예) CREATE TABLE post(id BIGINT AUTO_INCREMENT, title VARCHAR(30), contents VARCHAR(255), author_id BIGINT, 
                PRIMARY KEY (id), 
                FOREIGN KEY (author_id) REFERENCES author(id));
            - 참조되는 테이블에서 데이터의 수정/삭제 발생 시: ON DELETE나 ON UPDATE
                1. CASCADE
                    - 참조테이블의 key값을 삭제/수정하면 참조하는 테이블의 값도 같이 삭제/수정
                    - 예)
                        - SELECT CONSTRAINT_NAME 
                            FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE 
                            WHERE TABLE_NAME = 'post';
                        - -> post에 있는 author_id 값을 author 테이블에서 지워보는 것
                        - ALTER TABLE post 
                            DROP FOREIGN KEY post_ibfk_1, 
                            DROP INDEX author_id;
                        - ALTER TABLE post 
                            ADD CONSTRAINT post_author_fk FOREIGN KEY (author_id) REFERENCES author(id) 
                            ON DELETE CASCADE;
                        - -> 위와 같이 실행 => author(참조 source 테이블)의 값(row)을 지우게 되면 post(참조 갖고와서 만든 테이블)의 값(row)도 지워짐
                2. SET NULL
                    - 참조테이블의 key값을 삭제/수정하면 참조하는 테이블의 FK값이 NULL로 세팅됨
                    - 예)
                        - SELECT CONSTRAINT_NAME 
                            FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE 
                            WHERE TABLE_NAME = 'post';
                        - -> post에 있는 author_id 값을 author 테이블에서 지울 경우 post에서는 author_id값을 NULL로 바꾸는 것
                        - ALTER TABLE post 
                            DROP FOREIGN KEY post_author_fk, 
                            DROP INDEX  post_author_fk;
                        - ALTER TABLE post 
                            ADD CONSTRAINT post_author_fk 
                            FOREIGN KEY (author_id) 
                            REFERENCES author(id) 
                            ON DELETE SET NULL;
                        - -> 위와 같이 실행하면 author의 값을 지우게 되면 post의 FK값은 NULL이 됨
                3. RESTRICT
                    - FK로 잡은 테이블의 데이터가 남아 있으면, FK 대상 데이터 삭제/수정 불가
                    - 동작 옵션을 주지 않으면 기본은 RESTRICT
                4. soft delete 
                    - 데이터를 hard delete하지 않고 컬럼을 하나 더 만들어서 계정 삭제한 것들에는 다른 값을 주어서 계정 안 한 값들의 row만 불러오기
                    - -> soft delete / hard delete가 있는데 최근 추세는 대부분 soft delete를 하고 있음
                    - ORM이라는 기술을 통해, raw 쿼리를 쓰지 않는 게 추세
        - DEFAULT: 값 입력 안할 시 default로 자동입력
            - 예) ALTER TABLE author MODIFY COLUMN role ENUM('user', 'admin') NOT NULL DEFAULT 'user';
- 흐름제어
    - mysql에서는 if, else if문이 없고 그대신 CASE WHEN 구문이 있다
    - CASE
        - 문법:
            - CASE
                WHEN [compare_value] THEN result
                WHEN [compare_value] THEN result
                ELSE result
                END as 새컬럼명
        - CASE 와 END로 이루어져 있고 원하는 조건 내에 존재하지 않으면 ELSE 문을 타고 ELSE 문이 없을경우 NULL을 return
        - 예)
            - SELECT name, CASE WHEN grade >= 90 THEN 'A' WHEN grade >= 80 THEN 'B' WHEN grade >= 70 THEN 'C' ELSE 'F' END as my_grade from grade_table;
    - IF ()
        - 문법: IF(조건, b, c)
        - 만약 조건 참이면 b를 반환, 거짓이면 c를 반환
        - SELECT IF(0 < 1, 'yes', 'no');
        - 예)
            - SELECT name, email, IF (role IS NULL, 'user', role) as my_role from author;
    - IF NULL (a, b)
        - 만약 a의 값이 NULL이 아니면 a 그 자체를 반환, NULL이면 b를 반환
        - 예)
            - SELECT name, email, IFNULL (role, 'user') as FROM author;
- REGEXP 와 NOT REGEXP
    - 검색 패턴
    - 정규표현식을 토대로 패턴 연산 수행
    - SELECT * 
        FROM author 
        WHERE name REGEXP '^h|o$';
    - 'h'로 시작하거나 'o'로 끝나는
- JOIN: INNER JOIN 과 OUTER JOIN
    - 두 개 이상의 테이블의 컬럼을 조합하는 것
    - INNER JOIN - 가장 일반적인 형태
        - 테이블1 
            INNER JOIN 테이블2;
        - 테이블1과 테이블2의 공통변수가 있는 데이터만 조회
        - 이때 두 테이블의 자리를 바꿔도 결과는 똑같다
        - SELECT * FROM 테이블1 
            INNER JOIN 테이블2 
            ON 테이블1.컬럼명 = 테이블2.참조컬럼명;
            - 테이블1의 테이블1.컬럼명과 테이블2의 테이블2.참조컬럼명가 ON 조건에 일치하는 경우만 조회
                - 예) SELECT * 
                        FROM tableA 
                        INNER JOIN tableB 
                        ON tableA.ID = tableB.A_ID;
                - 예) SELECT * 
                        FROM tableA AS a 
                        INNER JOIN tableB AS b 
                        ON a.ID = b.a_id;
                        => AS 를 넣어주면 더 간결해짐
                - ***** ON 뒤에 오는 a.columna = b.columnb 의 두 값의 순서는 상관 없다
        - 예) author, post에 inner join 걸어서, author.name, author.email, post.title만을 조회해보자
            -   ```mysql
                SELECT author.name, author.email, post.title 
                FROM author 
                INNER JOIN post 
                ON author.id = post.author_id;
                ```
        - 예) author의 한 글쓴이가 post의 여러개의 글을 쓴 case insert
            -   ```mysql
                SELECT a.name, a.email, p.title 
                FROM author a 
                INNER JOIN post p 
                ON a.id = p.author_id;
                ```
    - OUTER JOIN 
        - LEFT OUTER JOIN (또는 그냥 LEFT JOIN) - RIGHT JOIN 보다 일반적으로 더 많이 사용
            - 테이블1 LEFT JOIN 테이블2;
            - 테이블1 기준으로 테이블2 결합 - 테이블1의 모든 데이터값은 다 나옴
            -   ```mysql
                SELECT * 
                FROM 테이블1 a 
                LEFT JOIN 테이블2 b 
                ON 테이블1.컬럼명 = 테이블2.참조컬럼명;
                ```
            - 테이블1의 테이블1.컬럼명과 테이블2의 테이블2.참조컬럼명가 ON 조건에 일치하는 경우만 조회;
                - 테이블1의 모든 데이터값은 다 나옴, 만약에 테이블2에 테이블2에 상당하는 값이 중복으로 나올 시 테이블1의 값이 중복된 만큼 출력됨
                - 예) 
                ```mysql
                    SELECT * 
                    FROM author a 
                    LEFT JOIN post b 
                    ON a.id = b.author_id 
                ```
                - author에 post left join해서 데이터 확인
        - RIGHT OUTER JOIN (또는 그냥 RIGHT JOIN)
            - 테이블1 RIGHT JOIN 테이블2;
            - 테이블2 기준으로 테이블1 결합
- UNION
    - SELECT 컬럼명1
        FROM 테이블명1
        UNION
        SELECT 컬럼명2
        FROM 테이블명2
    - 여러 개의 SELECT 문의 결과를 하나의 테이블이나 결과 집합으로 표현; 두 테이블을 하나의 테이블로 결합하는 키워드
    - 각각의 SELECT 문으로 선택된 컬럼의 개수와 타입이 모두 같아야 한다
    - UNION은 DISTINCT 키워드를 따로 명시하지 않아도 중복되는 레코드를 제거한다
    - UNION ALL 
        - 중복되는 레코드까지 모두 출력해주는 명령어




## SQL 함수
- SUM(): 행 안 값의 (또는 특정 컬럼의) 총합계
    - SELECT SUM(컬럼명) FROM 테이블명
    - 예) SELECT SUM(id) FROM author WHERE created_at BETWEEN '2023-06-01' AND '2023-07-01';
- AVG(): 행 안 값의 (또는 특정 컬럼의) 평균 값
- MIN(): 행 안 값의 (또는 특정 컬럼의) 최솟값
- MAX(): 행 안 값의 (또는 특정 컬럼의) 최댓값
- COUNT(): 조회 결과의 숫자값을 반환
    - SELECT COUNT(컬럼명) FROM 테이블명;       # COUNT와 (컬럼명) 사이 띄어쓰기 없어야 됨
    - SELECT COUNT(DISTINCT 컬럼명) FROM 테이블명;
    - 예) SELECT COUNT(*) FROM author WHERE created_at BETWEEN '2023-06-01' AND -2023-07-01';
- NOW(): 현재 시간 출력
    - SELECT NOW()
    - 보통 시스템이 어떤 시간대로 잡혀있는지 확인할 때 사용
    - 예) INSERT INTO author(email, created_at) values ('abc@naver.com', NOW());
        - 보통은 데이터를 입력하는 경우 위처럼 현재시간을 출력 안하고 TIMESTAMP를 사용
- CAST(): 타입 변환
    - CAST(a AS type)
    - a값을 type으로 변환
    - 특정한 type을 날짜값으로 변환할 때 많이 사용
    - 예) SELECT CAST(20200101 AS DATE);
    - 실습 예) author에 데이터 insert할 때 name, email, created_at insert; created_at은 cast를 사용하여 20220101030201 입력
        - INSERT INTO author (name, email, created_at) values ('michael', 'michael@hotmail.com', CAST(20220101030201 AS DATETIME));
- CONVERT()
    - 문자열을 날짜/시간으로 변환하는 데 사용
    - 예) CONVERT('2020-01-01', DATE); => 2020-01-01
- DATE_FORMAT(컬럼명, '%Y-%m-%d') 
    - 예) DATE_FORMAT(HIRE_YMD, '%Y-%m-%d') AS HIRE_YMD FROM DOCTOR => 이러면 시간 안나오고 날짜만 나옴
- INSTR('문자열', '추출 문자') 
    - 해당 문자열의 index를 추출
    - 예) SELECT INSTR('안녕하세요', '하세요');  => 3
- SUBSTR('문자열', '위치')
    - 해당 문자열의 위치부터 추출
    - 예) SELECT SUBSTR('반갑습니다', 3); => 습니다
- SUBSTRING('문자열', '위치', '길이')
    - 해당 문자열의 위치부터 길이까지 추출
    - 예) SELECT SUBSTRING('공부합시다', 3, 2); => 합시
- ROW_COUNT()
    - 가장 최근에 실행한 구문의 결과값 추출
- FLOOR()
    - 값보다 작은 정수 중 가장 큰 정수 추출; 소수점 이하 버림
- 년, 월, 일 추출
    - YEAR(), MONTH(), DAY()


## 용어
- table의 구조
    - 열(column) = 필드(field) = 속성    -> 컬럼 정보 바꿀께요~ 라고 하면 DDL로 alter table 하는 것
    - 행(row) = 데이터 = 레코드    -> 데이터 추가할께요~ 하고 하면 DML로 행을 추가하는 것
    - 값(value)
    - 키(key) = 행의 식별자
        - PRIMARY KEY (PK) = 테이블에 저장된 레코드를 고유하게 식별하는 컬럼/key
            - Not Null + Unique
            - PK 설정 시: ... , PRIMARY KEY (필드명)
                - 예) 
                ```mysql
                CREATE TABLE post(id BIGINT AUTO_INCREMENT, title VARCHAR(30), contents VARCHAR(255), author_id BIGINT, PRIMARY KEY(id));
                ```
        - FOREIGN KEY (FK) = 두 테이블을 연결하는 데 사용하는 컬럼/key
            - FK 설정 시: ... , FOREIGN KEY (필드명) REFERENCES 다른테이블명(다른테이블의필드명)
                - 예) CREATE TABLE post(id BIGINT AUTO_INCREMENT, title VARCHAR(30), contents VARCHAR(255), author_id BIGINT, PRIMARY KEY(id), FOREIGN KEY (author_id) REFERENCES author(id));
            - FK를 걸면 새테이블의 필드값은 reference하는 테이블의 필드에 있어야 입력 가능
                create table post(id BIGINT AUTO_INCREMENT, title VARCHAR(30), contents VARCHAR(255), author_id BIGINT, PRIMARY KEY(id));
                insert into post(title, contents, author_id) values ('hello', 'hello is ...', 100);
                insert into post(title, contents, author_id) values('hello', 'hello is ...', 1);
                >> Query OK --> 잘 들어감

                create table post(id BIGINT AUTO_INCREMENT, title VARCHAR(30), contents VARCHAR(255), author_id BIGINT, PRIMARY KEY(id), FOREIGN KEY (author_id) REFERENCES author(id));
                    - references에 author_id가 없으면 안 만들어진다!!!
                insert into post(title, contents, author_id) values ('hello', 'hello is ...', 100);
                >> ERROR ... foreign key constraint fails --> 에러 남: post의 author_id = 100 이 author의 id에 없기 때문
                insert into post(title, contents, author_id) values('hello', 'hello is ...', 1);
                >> Query OK --> 잘 들어감: post의 author_id = 1 이 author의 id에 있기 때문
- table의 특성
    - field = 열
    - type = 속성    -> 예) INT, VARCHAR(크기), etc.
    - NULL = 비어있는 값을 의미    -> YES 이면 비어있어도 된다는 의미
        - 컬럼 생성시 별다른 null관련 옵션을 주지 않으면 기본적으로 nullable.
- *
    - all
- 문자열은 쌍따음표가 아닌 따음표로 감싼다 (')
- #
    - 주석
    - 주석 뒤에 오는 문장은 컴퓨터가 안 읽음
    - 예) # select * from author
            select * from author where id >= 3 AND name = 'hong';
            >> 두번째 줄만 실행됨
- 명명
    - my_sql에서 두단어가 결합된 컬럼면을 지정할땐 보통 a단어_b단어   -> _ 사용


## MySQL 연산자
- mysql에서 없는 것
    - ==
    - ||는 다른 sql 언어 (oracle, postgres, sqlite, ansi sql 등)에서 문자열 결합(string concatenation) 역할을 함
        - mysql에서는 'logical or'이라는 뜻
- 산술 연산자
    - +, -, *, /, %(나머지), ** (제곱 연산; 표준X, 표준 제곱은 POWER)
    - NULL 값에 그 어떤 값을 +, -, *, / 해도 출력은 항상 NULL
    - ORACLE에서는 날짜 값에 자동 계산되서 적용되지만 MySQL에서는 날짜에 바로 적용 불가
        - 문법: interval (숫자) day
        - 예) SELECT created_at, date(created_at), DATE_ADD(date(created_at), interval 1 day) AS "+1 day"
                FROM board_main_post;
            - 출력: 
            ```
                +----------------------------+------------------+------------+
                | created_at                 | date(created_at) | +1 day     |
                +----------------------------+------------------+------------+
                | 2023-06-15 01:26:25.979626 | 2023-06-15       | 2023-06-16 |
                | 2023-06-15 02:41:40.198218 | 2023-06-15       | 2023-06-16 |
                | 2023-06-15 05:46:03.855349 | 2023-06-15       | 2023-06-16 |
                | 2023-06-15 06:21:47.528438 | 2023-06-15       | 2023-06-16 |
                +----------------------------+------------------+------------+
            ```
    - 예) SELECT name, price * 0.9 AS discount
            FROM Pearl;
        - 출력:  
        ``` 
            +-----------+----------+
            | name      | discount |
            +-----------+----------+
            | dwejibar  |    540.0 |
            | nugabar   |     NULL |
            | bibibik   |    540.0 |
            | worldcone |   1080.0 |
            +-----------+----------+
        ```
- 비교 연산자
    - >, <, >=, <=, =
    - !=(같지 않음), <>(같지 않음), 
    - !< (왼쪽 값이 오른쪽 값보다 작지 않음)
    - !> (왼쪽 값이 오른쪽 값보다 크지 않음)
    - <=> (= 와 같음, NULL 값에 대한 비교까지 수행; NULL-Safe, 즉 NULL도 하나의 값으로 인식하여 비교)
    - 예) SELECT * 
        FROM author 
        WHERE created_at > '2023-06-07';
    - 예) SELECT name, (price * 0.9)  AS discounted 
            FROM icecream
            WHERE price > 1000
        - 1000원보다 비싼 아이스크림들의 이름과 10% 할일 된 가격 출력
- 논리 연산자
    - ALL (다른 값들의 집합의 모든 값)
    - AND &&
    - OR || (다른 sql 언어에서는 문자열 결합이라는 사실 주의!)
    - NOT !
    - ANY (조건을 따르는 목록에 해당하는 값)
    - BETWEEN (포함 연산 방식, AND와 같이 사용)
        - 문법: BETWEEN a AND b
            - a, b 값 포함
        - 예) SELECT * 
                FROM author 
                WHERE created_at 
                BETWEEN '2023-06-01' AND '2023-07-01'; 
        - 예) SELECT * 
                FROM author 
                WHERE created_at > '2023-06-01' 
                AND created_at < '2023-07-01';
        - 10 < 컬럼명 AND 컬럼명 > 5 --> 파이썬처럼 5 < 값 < 10 안됨
    - EXISTS (행의 존재 여부)
    - IN (list), NOT IN (list)
        - list에 있는 값 중 하나만 일치해도 참
        - 문법: SELECT 컬럼명 FROM 테이블명 WHERE 컬럼명 IN (값1, 값2, 값3)
        - 예) SELECT * 
                FROM author 
                WHERE id IN (1, 3, 5);
        - 테이블 간 데이터값 공유 시;
        - SELECT * FROM 테이블명1 WHERE 테이블1의컬럼명1 IN (SELECT 테이블2의컬럼명1 from 테이블2);
        - 예) SELECT * 
                FROM author 
                WHERE id IN 
                    (SELECT author_id 
                        FROM post);
    - LIKE '비교문자열', NOT LIKE '비교문자열' 
        - 와일드카드 (%, _)를 사용하여 해당 문자열과 유사한 문자열을 찾으려 할 때 사용
        - 문법: SELECT 컬럼명 FROM 테이블명 WHERE 컬럼명 LIKE %(any)
        - 예) SELECT * 
                FROM author 
                WHERE name LIKE 'a%';
            - 'a'로 시작하는 것
        - 예) SELECT * 
                FROM author 
                WHERE name LIKE '%o';
            - 'o'로 끝나는 것
        - 예) SELECT * 
                FROM author 
                WHERE name LIKE '%e%';
            - 'e'가 중간이나 처음이나 끝에 있는 것
    - NOT (부정 연산자, 사용하려는 논리 연산자의 의미를 반전시킴, NOT EXISTS, NOT BETWEEN, NOT IN 등)
    - IS NULL, IS NOT NULL
        - 비어 있거나 비어있지 않거나
        - 주의: 컬럼 = NULL 하면 에러남
        - 예) SELECT * 
                FROM 테이블명 
                WHERE 컬럼명 IS NULL;
    - UNIQUE (유일성을 가지도록 구체화된 테이블의 모든 행 출력 - 중복 불허)
## MySQL 타입 (data type): 숫자
- 테이블의 타입 확인:
    - DESCRIBE 테이블명
    - 또는 DESC 테이블명
    - 출력 예)
    +--------+-------------+------+-----+---------+----------------+
    | Field  | Type        | Null | Key | Default | Extra          |
    +--------+-------------+------+-----+---------+----------------+
    | id     | int         | NO   | PRI | NULL    | auto_increment |
    | name   | varchar(30) | YES  |     | NULL    |                |
    | brand  | varchar(30) | YES  |     | NULL    |                |
    | flavor | varchar(30) | YES  |     | NULL    |                |
    | price  | bigint      | YES  |     | NULL    |                |
    +--------+-------------+------+-----+---------+----------------+
- 숫자 타입
    - 정수
        - TINYINT: -128 ~ 127범위, 1 Byte
            - UNSIGEND 타입 사용하여 양수만 표현 가능
            - TINYINT UNSIGNED 이렇게 사용한다면 255까지 사용가능
        - INT: 4 Byte
        - BIGINT: 8 Byte
    - 실수   
        - 고정 소수점 타입
            - DECIMAL (M, D)
                - 총자릿수 (정수부 + 소수부)를 의미하고, 65자리까지 표현가능
                - M은 정수부, D는 소수부 의미
                - 정확한 숫자 표현을 위해 사용
                - M, D 생략시에는 기본적으로 10, 0.
                - 예) practice_float table에 id1에 정수부 30자리, 소수부 4자리 decimal 타입으로 만들기
                    - CREATE TABLE practice_float(id1 DECIMAL(30, 4));
- 문자 타입
    - CHAR 와 VARCHAR
        - CHAR(M)
            - M은 문자열의 최대 길이를 의미
            - 고정 길이의 문자열; 0 - 255
            - 정해진 자릿수 문자에 제한을 두기 위해서 사용
        - VARCHAR(M)
            - M은 문자열의 최대 길이를 의미
            - 가변 길이의 문자열 ;0 - 65,535 Byte
            - M울 통해 별도의 길이 지정; 길이를 지정하더라도 실제 입력된 문자열의 길이만큼만 저장하고 사용가능
            - 일반적으로 가장 많이 사용
    - TEXT
        - TEXT
            - 가변 데이터; 0 - 65,535 Byte
            - VARCHAR와의 차이점: (면접에서 많이 물어봄 - index에 대해 아는 것을 설명하라)
                - VARCHAR는 
                    - 메모리를 적극적으로 사용하여 속도가 빠르나 메모리 낭비가 발생할 수 있다
                    - 최대 크기 지정
                    - index 사용 가능
                - TEXT는 
                    - disk에 저장해서 메모리 사용이 적어 조회 속도는 느리나 메모리는 효율적이다
                    - 최대 크기 지정 X
                    - index 사용의 어려움 (B-tree 인덱싱 불가, Full-Text 인덱스 가능)
        - BLOB
            - 다양한 크기의 바이너리 데이터를 저장할 수 있는 타입
            - 일반적으로 PNG와 같은 이미지파일을 저장할 때 지정하는 타입
        - ENUM
            - 특정 데이터들의 값(문자열 종류)을 지정하여 insert할 수 있도록 지정
            - 컬럼명 ENUM(데이터값1, 데이터값2, ...)
            - 예) ALTER TABLE author MODIFY COLUMN role ENUM('user', 'admin') NOT NULL DEFAULT 'user';
                - 이렇게 하면 NOT NULL옵션이 걸리고 role 입력을 안 한다면 DEFAULT로 'user'로 지정될 것임
- 날짜와 시간
    - DATE
        - 날짜를 저장할 수 있는 타입
        - YYYY-MM-DD
    - DATETIME(m)
        - 날짜와 함께 시간까지 저장, m 지정시 소수점 microseconds
        - YYYY-MM-DD HH-MM-SS
        - 가장 많이 사용
    - DATETIME DEFAULT CURRENT_TIMESTAMP
        - 현재시간을 default로 삽입하는 형식

## MySQL 날짜와 시간 함수
- 참조: https://dev.mysql.com/doc/refman/8.0/en/date-and-time-functions.html
- ORACLE에서는 날짜에 산술 연산 (+, - 등)이 바로 적용 가능하지만 MySQL에서는 산술 연산이 날짜에 맞춰서 적용이 안됨
- 날짜 전용 함수가 있다
    - ADDDATE(), ADDTIME(), DATE_ADD(), NOW(), TIMESTAMP(), 등등
    - 예) SELECT created_at, date(created_at), DATE_ADD(date(created_at), interval 1 day) AS "+1 day"
          FROM board_main_post;
    - 출력: +----------------------------+------------------+------------+
            | created_at                 | date(created_at) | +1 day     |
            +----------------------------+------------------+------------+
            | 2023-06-15 01:26:25.979626 | 2023-06-15       | 2023-06-16 |
            | 2023-06-15 02:41:40.198218 | 2023-06-15       | 2023-06-16 |
            | 2023-06-15 05:46:03.855349 | 2023-06-15       | 2023-06-16 |
            | 2023-06-15 06:21:47.528438 | 2023-06-15       | 2023-06-16 |
            +----------------------------+------------------+------------+
    - 예) SELECT created_at, date(created_at), DATE_ADD(date(created_at), interval 1 year) AS "+1 year"
          FROM board_main_post;
    - 출력: +----------------------------+------------------+------------+
           | created_at                 | date(created_at) | +1 year    |
           +----------------------------+------------------+------------+
           | 2023-06-15 01:26:25.979626 | 2023-06-15       | 2024-06-15 |
           | 2023-06-15 02:41:40.198218 | 2023-06-15       | 2024-06-15 |
           | 2023-06-15 05:46:03.855349 | 2023-06-15       | 2024-06-15 |
           | 2023-06-15 06:21:47.528438 | 2023-06-15       | 2024-06-15 |
           +----------------------------+------------------+------------+

## 이미지/영상을 db에 저장하는 방법:
1. path만 넣어두고, 실제 저장은 disk
2. blob: 이미지/영상 파일 그 자체를 2진법체계로 변환시킨 바이너리 파일 형태로 db에 통째로 저장

## DBMS (Data Management System)
- NOTE: 
    - 쿼리문은 관용적으로 대문자 사용(대소문자구문X)
    - 테이블명, 컬럼명, 변수명 (데이터)은 대소문자를 구분함
    - 관계형데이터베이스 (RDB) 라고도 부른다: 서로 관계를 갖는 데이터들의 집합
- DDL (Data Definition Lang) 데이터 정의어: 테이블/컬럼 정의/변경하는 명령어
    - CREATE TABLE 테이블명(필드이름1 필드타입1, 필드이름2 필드타입2, ...)
        - 생성
        - 예) CREATE TABLE author(id INT, name VARCHAR(10), email VARCHAR(20));
    - ALTER TABLE 테이블명
        - 수정 (TABLE 구성변경)
        - 테이블명 변경
            - ALTER TABLE 테이블명 RENAME 새로운테이블명;
            - 예) ALTER TABLE author RENAME member;
        - 컬럼명 변경
            - ALTER TABLE 테이블명 RENAME COLUMN 변경대상컬럼명 TO 새로운컬럼명;
            - 예) ALTER TABLE author RENAME COLUMN pw TO password;
        - 컬럼 추가
            - ALTER TABLE 테이블명 ADD COLUMN 새로운필드명 필드타입;
            - 예) ALTER TABLE author ADD COLUMN updated_at DATETIME;
        - 컬럼 옵션 덮어 붙여쓰기
            - ALTER TABLE 테이블명 MODIFY COLUMN 필드명 필드타입 추가옵션
            - 예) ALTER TABLE author MODIFY COLUMN created_at DATETIME DEFAULT CURRENT_TIMESTAMP;
        - 컬럼 삭제
            - ALTER TABLE 테이블명 DROP COLUMN 삭제할컬럼명;
    - DROP TABLE 테이블명
        - 삭제
    - RENAME
    - TRUNCATE
        - 테이블 안 모든 데이터를 한꺼번에 삭제
        - 문법: TRUNCATE TABLE 테이블명
        - 삭제 후 복구 불가능
        - DELETE보다 속도 더 빠름
- DML (Data Manipulation Lang) 데이터 조작어: 데이터 조회/추가/수정/삭제
    - SELECT 
        - 조회 - 조건이 없으면 모든 data 조회
        - 문법: SELECT [ALL/DISTINCT] 컬럼/필드명 FROM 테이블명 WHERE 조건
        - 컬럼/필드명을 명시해도 되고, * 을 통해 모든 컬럼 조회도 가능
        - * 
            - 테이블의 모든 데이터 출력
            - 예) SELECT * 
                    FROM author;
        - ALL
            - 중복 데이터 모두 출력 (default)
            - 문법: SELECT [ALL] 컬럼/필드명 FROM 테이블명
            - 예) SELECT [ALL] name, email 
                    FROM author;
        - DISTINCT 
            - 중복되는 값 제거
            - 문법: SELECT DISTINCT 컬럼명1 [, 컬럼명2, 컬럼명3 ...] FROM 테이블명;
            - DISTINCT 키워드는 SELECT 뒤와 첫 컬럼의 앞에 위치해야 함
            - NULL 값도 하나의 값으로 간주함
                - MySQL에서 NULL값은 가장 작은 값으로 간주됨 -> ORDER BY 했을 때 가장 작은 값으로 간주됨
            - 예) SELECT DISTINCT name
                    FROM author;
                - 중복된 name들 제거되고 하나씩만 반환
            - 컬럼이 하나 이상인 경우 모든 컬름들의 조합을 출력
            - 예) SELECT DISTINCT name, email 
                    FROM author;
                - name과 email의 가능한 조합 반환
        - LIMIT 숫자 옵션을 주면 제한된 데이터만 조회
            - 문법: LIMIT n;
            - 예) SELECT name, email 
                    FROM author 
                    LIMIT 1;
            - 주의: 다른 database system들은 다른 키워드를 사용한다
                - SQL Server / MS Access:
                    - 문법: SELECT TOP 숫자 컬럼명
                            FROM 테이블명;
                    - percent도 가능: SELECT TOP 숫자 PERCENT 컬럼명
                                        FROM 테이블명;
                - MySQL:
                    - 믄밥: SELECT 컬럼명
                            FROM 테이블명
                            WHERE 조건
                            LIMIT 숫자;
                - Oracle 12:
                    - 문법: SELECT 컬럼명
                            FROM 테이블명
                            ORDER BY 컬럼명
                            FETCH FIRST 숫자 ROWS ONLY;
                    - percent도 가능: SELECT 컬럼명
                                     FROM 테이블명
                                        ORDER BY 컬럼명
                                        FETCH FIRST 숫자 PERCENT ROWS ONLY;
                - Older Oracle:
                    - 문법: SELECT 컬럼명
                            FROM 테이블명
                            WHERE ROWNUM <= 숫자;
                    - 문법 (with ORDER BY): SELECT 컬럼명
                                            FROM (SELECT 컬럼명 FROM 테이블명 ORDER BY 컬럼명)
                                            WHERE ROWNUM <= 숫자;

        - WHERE ~
            - 조건 걸기, 특정 조건을 만족하는 데이터 한정 출력
            - 문법: SELECT 컬럼명 FROM 테이블명 WHERE 조건
                - WHERE 컬럼명 (연산자) 컬럼값
            - 예) ... WHERE 컬럼명 = 컬럼값
        - ORDER BY
            - 선택한 결과의 정렬 (항상 조건 다 걸고 마지막에 붙임)
            - 문법: ORDER BY 컬럼명 [ASC/DESC]
            - SELECT 문으로 선택한 결과를 ORDER BY 절을 사용하여 정렬
            - 기본 설정은 ASC
            - 여러 컬럼의 데이터를 쉼표(,)를 사용하여 한 번에 정렬 가능
            - 예) SELECT * 
                    FROM author 
                    ORDER BY id DESC;
                - id를 기준으로 내림차순 정렬
            - 예) SELECT * 
                    FROM author 
                    ORDER BY name [ASC];
                - name을 기준으로 오름차순 정렬 [ASC]는 optional
            - 예) SELECT name, email
                    FROM author
                    ORDER BY 2;
                - 컬럼이 하나 이상일 경우 처음 컬럼부터 번호가 매겨진다 (name = 1, email = 2, ...)
                - ORDER BY 2에서 '2'는 SELECT 뒤 두번째로 명시된 컬럼인 'email'을 가리킨다
                - 그러므로 email을 기준으로 오름차순 정렬
            - ORDER BY절 생략시 PK를 기준으로 정렬하여 결과값 반환
            - ORDER BY절에는 SELECT절에 없는 컬럼도 사용 가능
            - 예) SELECT name 
                    FROM author 
                    ORDER BY email;
                - email을 기준으로 author 테이블의 name들이 오름차순 정렬로 출력
            - 예제) name만을 기준으로 오름차순 정렬
                - SELECT * FROM author ORDER BY name ASC;
            - 예제) name, email 기준으로 오름차순 정렬; name 중복일경우, email을 기준으로 정렬하게 되는 것
                - SELECT * FROM author ORDER BY name, email;
            - 예제) name은 오름차순, email은 내림차순으로 정렬
                - SELECT * FROM author ORDER BY name ASC, email DESC;
        - GROUP BY
            - 선택된 레코드의 집합을 툭정조건으로 그룹화한 결과 집합
            - 데이터의 값을 집계, 주로 집계 함수와 같이 사용
            - 집계 함수 예) SUM(), COUNT(), AVG(), MIN(), MAX() 등
            - 예) 작가가 쓴 글의 갯수를 알아보기; 이름, 이메일, 글 쓴 갯수 출력
                - SELECT a.name AS author_name, a.email, 
                            COUNT(p.author_id) AS 'post_count' 
                    FROM author a 
                    LEFT JOIN post p ON a.id = p.author_id 
                    GROUP BY a.email;
            - 예) sales테이블에서 지점별(지점ID) 평균매출 구하기
                - SELECT branch_id, AVG(amount) AS average 
                    FROM sales 
                    GROUP BY branch_id;
        - GROUP BY + HAVING 조건절
            - 구문 전체에 대한 조건절은 WHERE ~ 이고 그룹화된 데이터에 대한 조건절은 HAVING ~
            - 예) author에서 중복되는 이름과 중복되는 횟수 조회
                - SELECT name, COUNT(*) AS COUNT 
                    FROM author 
                    GROUP BY name 
                    HAVING COUNT(*) > 1;
            - 예) 2022년 한해동안 지점별 총매출을 구하라. 정렬은 금액 내림차순. 총매출이 2000 이상인 지점만 출력. 출력 - 지점명, 총매출액
                - SELECT b.name, SUM(s.amount)
                    FROM branches AS b 
                    INNER JOIN sales AS s 
                    ON b.id = s.branch_id 
                    WHERE s.created_date >= '2022-01-01' 
                    AND s.created_date < '2023-01-01' 
                    GROUP BY b.name 
                    HAVING SUM(s.amount) >= 2000.00;
        - AS
            - 테이블과 필드에 임시로 별칭(alias)을 부여하고, 해당 별칭을 SELECT문에서 사용
            - AS 라는 키워드를 줘서 별칭(alias)를 이용한 처리 
            - 문법: SELECT 컬럼명 [AS] 별칭 FROM 테이블이름;
            - 생략 가능
            - 예) SELECT COUNT(DISTINCT name) AS count_name 
                    FROM author;
            - 예) SELECT COUNT(DISTINCT name) count_name 
                    FROM author;
            - 출력되는 별칭은 WHERE, FROM, GROUP BY, HAVING, ORDER BY 뒤에는 사용하지 않는다; 임시로 부여한 것이기 때문에 명령어 안에서는 사용되지 않는다
            - 별칭이 공백, 특수문자 등 포함 시 큰 따옴표("") 사용
                - 공백이나 특수문자 포함 안해도 "" 사용해도 됨
            - 예) SELECT name AS "작가 이름",
                            date AS "날짜!"
                            content AS "내용"
                    FROM author
    - INSERT INTO 
        - 테이블 한 건의 레코드 추가
            - 문법1: INSERT INTO 테이블명 (필드명1, 필드명2, 필드명3 ...) VALUES (데이터값1, 데이터값2, 데이터값3 ...);  -> 필드타입 넣을 필요 없음
                - 예) INSERT INTO author(id, name, email) values (1, 'leo', 'leo@gmail.com');
                - 컬럼 순서는 실제 테이블의 컬럼 순서와 무관
                - 정의하지 않은 컬럼은 NULL이 입력됨
            - 문법2: INSERT INTO 테이블명 VALUES (전체 컬럼의 value_list)
                - 예) INSERT INTO Pearl VALUES (13, 'merona', 'binggrae', 'melon', 600)
                - 전체 컬럼의 모든 값을 순서대로 입력해야 됨
                - 빈 값은 NULL 로 입력
                    - 주의: MySQL에서는 안 되지만 ORACLE에서는 NULL 또는 작은 따옴표 ('')로 입력 가능 (**주의: 작은 따옴표 안에 공백이 있으면 안 됨 - 예) ' '는 하나의 공백 값으로 인식)
        - 여러 건 입력 시
            - 문법1: INSERT INTO 테이븖명 (컬럼1, 컬럼2, 컬럼3)
                        VALUES (데이터1, 데이터2, 데이터3),
                                (데이터4, 데이터5, 데이터6),
                                (데이터7, 데이터8, 데이터9);
            - 문법2: INSERT INTO 테이블명 VALUES (데이터1, 데이터2, 데이터3),
                                                (데이터4, 데이터5, 데이터6),
                                                (데이터7, 데이터8, 데이터9);
        - 문자 또는 날짜 값은 작은 따옴표로 묶음 (숫자 데이터는 작은 따옴표 없이 사용)
    - UPDATE 
        - 데이터 수정
        - 문법: UPDATE 테이블명 SET 필드명1 = 수정값1, 필드명2 = 수정값2 WHERE 필드명 = 데이터값;
        - 조건이 없으면 다 바꿈 (조심해야 함)
        - 예) UPDATE author 
                SET email = 'pearl.ahn@gmail.com' 
                WHERE id = '2';
    - DELETE
        - 데이터 삭제 (테이블에 존재하는 전체 레코드 삭제)
        - 문법: DELETE FROM 테이블명 WHERE 필드명 = 데이터값;
        - 한 row 전체를 지우기 때문에 컬럼명이나 *를 지정해줄 필요 없다
        - 삭제 후 복구 가능 (TRUNCATE는 복구 불가)
        - 조건을 빼버리면 테이블 전체를 지움 (조심해야 됨)
        - 예) DELETE FROM author 
                WHERE id >= 10;
- DCL (Data Control Lang) 데이터 제어어: 사용자 권한 관리 (계정 생성 후 권한부여)
    - GRANT (권한 부여)
    - REVOKE (권한 취소)
- + TCL (Transaction Control Lan) 트랜젝션 제어어: 변경 내용을 확정/취소
    - COMMIT
    - ROLLBACK

insert into codingtest (writer_id, price, status) values ('miyeon89', 1500, 'DONE');

## 단축키
- 터미널에서 쿼리문 종료 단축키: Ctrl + C

## 순서 예:
- SELECT NAME, COUNT(*) AS COUNT 
  FROM ANIMAL_INS 
  WHERE NAME IS NOT NULL
  GROUP BY NAME
  HAVING COUNT(*) > 1
  ORDER BY NAME ASC;

## Docker 마이그레이션
- python3 manage.py makemigrations --> DDL문생성
- python3 manage.py migrate --> DDL문 생성 후 반영

## 한국어 깨질 때
- 명령 프롬프트에서
    - status; 입력 후 사용하고 있는 database의 상태 확인
    - 혹은 show variables like '%c'; 입력 후 다음을 확인:
    ```
        +--------------------------+--------+
        |Variable_name             |Value   |
        |--------------------------|--------|
        | character_set_client     | utf8   |
        | character_set_connection | utf8   |
        | character_set_database   | utf8   |
        | character_set_filesystem | binary |
        | character_set_results    | utf8   |
        | character_set_server     | utf8   |
        | character_set_system     | utf8   |
        +--------------------------+--------+
    ```
- 만약에 위와 같이 설정되어 있지 않은 경우:
    - 명령 프롬프트에서 다음과 같이 설정:
        - SET character_set_client = 'utf8';
        - SET character_set_connection = 'utf8';
        - SET character_set_database = 'utf8';
        - SET character_set_results = 'utf8';
        - SET character_set_server = 'utf8';
        - SET character_set_system = 'utf8';